In [7]:
# Exercício 1 -> Previsão de Consumo de Combustível

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Criar dataset simulado
np.random.seed(42)
n = 100

dados_combustivel = pd.DataFrame({
    "carga_ton": np.random.uniform(2, 15, n),
    "tempo_trajeto": np.random.uniform(10, 60, n),
    "velocidade_media": np.random.uniform(20, 80, n),
    "tipo_piso": np.random.choice(["asfalto", "brita", "terra"], n, p=[0.5, 0.3, 0.2])
})

# Codificação temporária para simular a variável-alvo com base em fatores
piso_dummies = pd.get_dummies(dados_combustivel['tipo_piso'], prefix='tipo_piso', drop_first=True)

# Criar a variável-alvo (consumo_litros)
# Consumo aumenta com a carga e tempo, diminui com a velocidade, e é maior em brita e terra.
dados_combustivel["consumo_litros"] = (
    0.8 * dados_combustivel["carga_ton"] +
    0.05 * dados_combustivel["tempo_trajeto"] +
    -0.03 * dados_combustivel["velocidade_media"] +
    3 * piso_dummies.get("tipo_piso_brita", 0) +
    5 * piso_dummies.get("tipo_piso_terra", 0) +
    np.random.normal(0, 1.5, n) # Ruído
)

# 2. Pré-processar os dados
# Codificação One-Hot para 'tipo_piso'
dados_combustivel = pd.get_dummies(dados_combustivel, columns=['tipo_piso'], drop_first=True)

# Variáveis Preditivas (X) e Variável Alvo (y)
X = dados_combustivel.drop("consumo_litros", axis=1)
y = dados_combustivel["consumo_litros"]

# 3. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Lista de modelos a serem comparados
modelos = {
    "Regressão Linear": LinearRegression(),
    "Árvore de Regressão": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42)
}

resultados_combustivel = {}

# 4. Treinar e avaliar os modelos
print("Exercício 1: Previsão do Consumo de Combustível\n")
for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # Cálculo das métricas
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    resultados_combustivel[nome] = {"EQM": mse, "EAM": mae, "R2": r2}

# 5. Exibir as métricas
df_resultados_combustivel = pd.DataFrame(resultados_combustivel).T

print(df_resultados_combustivel.sort_values(by='R2', ascending=False))
print("\nO modelo Random Forest obteve o melhor desempenho (maior R2 e menores EQM/EAM).")


print("-" * 50)

Exercício 1: Previsão do Consumo de Combustível

                               EQM       EAM        R2
Regressão Linear          1.843035  1.080262  0.883393
Random Forest Regressor   3.844721  1.582299  0.756748
Árvore de Regressão      10.436583  2.306107  0.339688

O modelo Random Forest obteve o melhor desempenho (maior R2 e menores EQM/EAM).
--------------------------------------------------
